# Big table

Median and UI intervals for:

* Within diseases, proportions across regions

In [1]:
library(data.table)

In [2]:
sms <- list.files("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/")

In [3]:
dis <- as.numeric(substr(sms,25,nchar(sms)-4))
dis

[1]  0  1 10 12 13 14 15 16 17 18 19  2 20 22 23 24 25 26  3  4  5  6  7  8  9

In [4]:
Mgbd <- read.table("../Data/27_gbd_groups.txt")

In [5]:
k <- 1
DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))
regs <- sort(unique(DF$Region))
regs <- regs[regs!="All"]

In [6]:
data_f <- data.frame()

for(k in dis[dis!=0]){

    DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))

    DFr <- DF[DF$Region%in%regs & DF$Dis == "dis",]
    DFr$RCTs_all <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$RCTs_NHI <- rep(DF$RCTs[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))
    DFr$Patients_all <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="All"],each=length(regs))
    DFr$Patients_NHI <- rep(DF$Patients[DF$Dis=="dis" & DF$Region=="Non-HI"],each=length(regs))

    df <- data.frame(cbind(regs,as.character(Mgbd$x[k]),
        do.call('rbind',by(DFr[DFr$RCTs_all!=0,],
                           DFr$Region[DFr$RCTs_all!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$Patients_all!=0,],
                           DFr$Region[DFr$Patients_all!=0],
                           function(x){100*quantile(x$Patients/x$Patients_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$RCTs_NHI!=0,],
                           DFr$Region[DFr$RCTs_NHI!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_NHI,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$Patients_NHI!=0,],
                           DFr$Region[DFr$Patients_NHI!=0],
                           function(x){100*quantile(x$Patients/x$Patients_NHI,probs=c(0.025,0.5,0.975))})))
        )

    names(df) <- c("Region","Disease",
               paste(paste("Prop_all","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_all","Patients",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","Patients",sep="_"),c("low","med","up"),sep="_"))

    data_f <- rbind(data_f,df)
}

In [7]:
#All diseases
k <- 0
    DF <- fread(paste(c("/media/igna/Elements/HotelDieu/Cochrane/MappingRCTs_vs_Burden/Replicates/Metrics_over_repl/Metrics_over_replicates_",
                    as.character(k),".txt"),collapse=""))

    DFr <- DF[DF$Region%in%regs,]
    DFr$RCTs_all <- rep(DF$RCTs[DF$Region=="All"],each=length(regs))
    DFr$RCTs_NHI <- rep(DF$RCTs[DF$Region=="Non-HI"],each=length(regs))
    DFr$Patients_all <- rep(DF$Patients[DF$Region=="All"],each=length(regs))
    DFr$Patients_NHI <- rep(DF$Patients[DF$Region=="Non-HI"],each=length(regs))

    df <- data.frame(cbind(regs,"All",
        do.call('rbind',by(DFr[DFr$RCTs_all!=0,],
                           DFr$Region[DFr$RCTs_all!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$Patients_all!=0,],
                           DFr$Region[DFr$Patients_all!=0],
                           function(x){100*quantile(x$Patients/x$Patients_all,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$RCTs_NHI!=0,],
                           DFr$Region[DFr$RCTs_NHI!=0],
                           function(x){100*quantile(x$RCTs/x$RCTs_NHI,probs=c(0.025,0.5,0.975))})),
        do.call('rbind',by(DFr[DFr$Patients_NHI!=0,],
                           DFr$Region[DFr$Patients_NHI!=0],
                           function(x){100*quantile(x$Patients/x$Patients_NHI,probs=c(0.025,0.5,0.975))})))
        )

    names(df) <- c("Region","Disease",
               paste(paste("Prop_all","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_all","Patients",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","RCTs",sep="_"),c("low","med","up"),sep="_"),
               paste(paste("Prop_NHI","Patients",sep="_"),c("low","med","up"),sep="_"))

    data_f <- rbind(data_f,df)

In [8]:
rownames(data_f) <- NULL

In [9]:
data_f[data_f$Region%in%c("High-income","Non-HI"),grep("NHI",names(data_f))] <- NA

In [10]:
head(data_f)

,Region,Disease,Prop_all_RCTs_low,Prop_all_RCTs_med,Prop_all_RCTs_up,Prop_all_Patients_low,Prop_all_Patients_med,Prop_all_Patients_up,Prop_NHI_RCTs_low,Prop_NHI_RCTs_med,Prop_NHI_RCTs_up,Prop_NHI_Patients_low,Prop_NHI_Patients_med,Prop_NHI_Patients_up
1,"Central Europe, Eastern Europe, and Central Asia",Tuberculosis,3.69003740785565,5.80474934036939,8.13568226393947,0.366473691455004,0.839127809160704,2.83936826324646,5.08474576271187,8.01526717557252,11.486262637802,0.380957658815296,0.889686757284583,3.33797750372922
2,High-income,Tuberculosis,25.2298560242485,32.6086956521739,43.75,2.0993861537554,5.01358306762257,20.7015918123193,NA,NA,NA,NA,NA,NA
3,Latin America and Caribbean,Tuberculosis,6.90279038112523,9.6969696969697,12.5560538116592,0.984953513555074,7.43862809488715,24.4784120543199,9.86401964418087,13.3333333333333,17.0268295534253,1.0378086262667,7.78820711422704,27.9627474021534
4,Non-HI,Tuberculosis,62.0564190128775,72.9805013927577,80.0802811870901,79.2984081876807,94.9864169323774,97.9006138462446,NA,NA,NA,NA,NA,NA
5,North Africa and Middle East,Tuberculosis,3.59039767997637,5.86510263929619,8.48484848484849,0.186777744586216,0.634283140727821,2.09269030195581,4.84384164222874,8.05369127516778,12.5,0.192695325477562,0.670666221682141,2.42120570680917
6,South Asia,Tuberculosis,9.76133765243902,13.2315521628499,16.4971649484536,1.54968676109187,3.18532882501455,10.4758002001611,14.1666666666667,18.232044198895,22.2222222222222,1.62335420612825,3.36210946299576,12.183418694834


In [11]:
write.table(data_f,"../Data/RCTs_and_Patients_prop_among_all_and_HI_median_UI_across_regions_per_disease.txt")